In [53]:
import pymysql
import pandas as pd
import glob 
import pandas as pd
import os
con =pymysql.connect(host='ip', user='id', password='pw', db='encore6', charset='utf8')
cur = con.cursor()
# cur.execute("show tables")
# CycleRentUse_df = pd.read_sql_query("select * from CycleRentUse", con=con)

In [54]:
csv_test = pd.read_csv('./hire_point.csv')

In [55]:
csv_test.head(1)

,Unnamed: 0,STA_LOC,RENT_ID,RENT_NO,RENT_NM,RENT_ID_NM,HOLD_NUM,STA_ADD1,STA_ADD2,STA_LAT,STA_LONG,START_INDEX,END_INDEX,RNUM
0,0,마포구,ST-10,108,서교동 사거리,108. 서교동 사거리,12.0,서울특별시 마포구 양화로 93,427,37.552746,126.918617,0,0,1


In [56]:
csv_test.columns

Index(['Unnamed: 0', 'STA_LOC', 'RENT_ID', 'RENT_NO', 'RENT_NM', 'RENT_ID_NM',
       'HOLD_NUM', 'STA_ADD1', 'STA_ADD2', 'STA_LAT', 'STA_LONG',
       'START_INDEX', 'END_INDEX', 'RNUM'],
      dtype='object')

In [57]:
type(csv_test)

pandas.core.frame.DataFrame

In [58]:
data = csv_test[['Unnamed: 0', "STA_LOC", "STA_LAT", "STA_LONG"]]

In [59]:
data.head(1)

,Unnamed: 0,STA_LOC,STA_LAT,STA_LONG
0,0,마포구,37.552746,126.918617


In [60]:
sql= """CREATE Table hire_point
(
    `Unnamed: 0` INT,
    `STA_LOC` VARCHAR(30),
    `STA_LAT` FLOAT,
    `STA_LONG` FLOAT
)"""
cur.execute(sql)
con.commit()

In [63]:
insert_sql="INSERT INTO hire_point VALUES (%s, %s, %s, %s)"

In [64]:
data.columns

Index(['Unnamed: 0', 'STA_LOC', 'STA_LAT', 'STA_LONG'], dtype='object')

In [65]:
data.apply(lambda x : cur.execute(insert_sql,x.tolist()),axis=1)

0       1
1       1
2       1
3       1
4       1
       ..
3074    1
3075    1
3076    1
3077    1
3078    1
Length: 3079, dtype: int64

In [66]:
con.commit()

In [69]:
#년도 그룹, hire_point violat_accidnet, cyclyrentuse
hire_point = pd.read_sql_query("select * from hire_point", con=con)
violation_accident = pd.read_sql_query("select * from violation_accident", con=con)
cyclyrentuse = pd.read_sql_query("select * from CycleRentUse", con=con)
# 지자체	 자전거사고건수	자전거이용건수	위도	경도

/tmp/ipykernel_21833/228760671.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  hire_point = pd.read_sql_query("select * from hire_point", con=con)
/tmp/ipykernel_21833/228760671.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  violation_accident = pd.read_sql_query("select * from violation_accident", con=con)
/tmp/ipykernel_21833/228760671.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  cyclyrentuse = pd.read_sql_query("select * from CycleRentUse", con=con)


In [120]:
cyclyrentuse.head(1)

,Rent_Date,Rent_Number,Rent_Count
0,2019-01,3,64


In [82]:
# left join으로 어디서 빌렸는지를 기준으로 통합한다.
cyclyrentuse_hire_point = pd.merge(hire_point, cyclyrentuse,  left_on=['Unnamed: 0'], right_on=['Rent_Number'], how='left')
# pd_rt1_left = pd.merge(df1, df2, left_on=['orderNumber'], right_on=['orderNumber'], how='left')

In [87]:
# nan값 내리
new_data = cyclyrentuse_hire_point.dropna()

In [90]:
# sta_loc 구별로 묶어서 rent_count sum하기
new_data_group = new_data.groupby(["STA_LOC"], as_index=False)[["Rent_Count"]].sum()
# CycleRentUse_year_df=CycleRentUse_df.groupby(["Rent_Date"],as_index=False)[['Rent_Count']].sum().sort_values(by=["Rent_Date"])

In [92]:
new_data_group.head(1)

,STA_LOC,Rent_Count
0,강남구,4376602.0


In [99]:
len(new_data_group)

25

In [96]:
temp_vio = violation_accident.groupby(["JIJACE_NM"], as_index=False)[["ACC_CNT"]].sum()

In [98]:
len(temp_vio)

25

In [102]:
total =  pd.concat([new_data_group, temp_vio], axis = 1)

In [108]:
new_total = total.drop(["JIJACE_NM"],axis=1)

In [115]:
new_total.head(1)

,STA_LOC,Rent_Count,ACC_CNT
0,강남구,4376602.0,18462


In [118]:
import folium
import json
import requests
map = folium.Map(location=[37.5502, 126.982], zoom_start=15, tiles="cartodb positron")
geo_str = requests.get("https://t1.daumcdn.net/cfile/tistory/272C224C58B4BD540B").json()
# map
m = folium.Choropleth(geo_str, 
                  data = new_total,
                  columns=['STA_LOC', 'ACC_CNT'],
                  fill_color="PuRd",
                  key_on="feature.id").add_to(map)
                  

In [119]:
#구에 따른 자전가 사고 사건의 개수
map

In [95]:
violation_accident.head(1)

,JIJACE_NM,CHISA,ACC_HUM,SIDO_CD,GUBUN_NM,SLTWD_CNT,SERI_ACC,SIDO_NM,AC_MODE,CRT_JIJACE_CD,...,TRK_CNT,DEATH_CNT,BASE_CODE,WND_CNT,DEATH_ACC,GUBUN,SERI_CNT,SLTWD_ACC,YEAR,BASE_CATEGORY
0,강남구,0.0,111,11000,승용,70,27,서울,A,11680,...,0,0,01,8,0,01,33,42,2018,위반


In [93]:
# hire_point.head(1)

In [94]:
# cyclyrentuse_group=cyclyrentuse.groupby(["Rent_Date"],as_index=False)[['Rent_Count']].sum().sort_values(by=["Rent_Date"])